In [1]:
import os
import numpy as np
import torch
from torchvision import datasets
from torch.utils.data import Dataset
import copy

/usr/local/home/sgchr/anaconda3/envs/pyt/lib/python3.8/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: /usr/local/home/sgchr/anaconda3/envs/pyt/lib/python3.8/site-packages/torchvision/image.so: undefined symbol: _ZN5torch3jit17parseSchemaOrNameERKNSt7__cxx1112basic_stringIcSt11char_traitsIcESaIcEEE
  warn(f"Failed to load image Python extension: {e}")


In [2]:
def get_CIFAR10(path):
    data_tr = datasets.CIFAR10(path + '/CIFAR10', train=True, download=True)
    data_te = datasets.CIFAR10(path + '/CIFAR10', train=False, download=True)
    X_tr = data_tr.data
    Y_tr = torch.from_numpy(np.array(data_tr.targets))
    X_te = data_te.data
    Y_te = torch.from_numpy(np.array(data_te.targets))
    return X_tr, Y_tr, X_te, Y_te

In [3]:
X_tr, Y_tr, X_te, Y_te = get_CIFAR10('data')

Files already downloaded and verified
Files already downloaded and verified


In [4]:
def decrease_dataset(X_tr, Y_tr):
    new_size = 50
    masks = [np.zeros(len(Y_tr), dtype = 'int') for i in range(10)]
    for i in range(len(Y_tr)):
        masks[Y_tr[i].item()][i] = 1

    # creating new_Xtr
    new_Xtr = []
    for i in masks:
        var = np.random.choice(np.arange(len(X_tr), dtype=int)[i == 1], int(new_size/10), replace=False)
        new_Xtr.append(X_tr[var])

    new_Xtr = np.array(new_Xtr).reshape(new_size, 32, 32, 3)

    # creating new_Ytr
    new_Ytr = np.zeros(new_size, dtype = 'int')
    slce = int(new_size/10)
    for i in range(10):
        new_Ytr[int(slce-new_size/10):slce] = i
        slce += int(new_size/10)
    return new_Xtr, torch.from_numpy(new_Ytr)

In [5]:
new_Xtr, new_Ytr = decrease_dataset(X_tr, Y_tr)

In [6]:
def relabel(new_X, new_Size = 50):
    img_per_cls = new_Size/10
    class_to_swap = 3
    long = int(img_per_cls * class_to_swap)
    cls_to_swp_with = 7
    long_two = int(img_per_cls * cls_to_swp_with)
    new_X[[long,long_two],:,:,:] = new_X[[long_two, long],:,:,:]
   
    return new_X

In [3]:
a = np.arange(50)
print(a)

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49]


In [5]:
b = relabel(a)

In [7]:
c = relabel(new_Xtr)

In [9]:
c

array([[[[ 25,  86,  26],
         [ 26,  86,  27],
         [ 27,  87,  28],
         ...,
         [ 29,  89,  30],
         [ 28,  88,  29],
         [ 28,  88,  29]],

        [[ 25,  84,  26],
         [ 26,  85,  27],
         [ 27,  86,  28],
         ...,
         [ 28,  87,  29],
         [ 27,  86,  28],
         [ 27,  86,  28]],

        [[ 25,  85,  26],
         [ 26,  85,  27],
         [ 27,  86,  28],
         ...,
         [ 27,  86,  28],
         [ 26,  85,  27],
         [ 26,  85,  27]],

        ...,

        [[ 10,  58,  11],
         [ 11,  58,  12],
         [ 11,  58,  12],
         ...,
         [ 12,  58,  12],
         [ 12,  58,  12],
         [ 11,  57,  11]],

        [[  9,  56,  10],
         [  9,  56,  10],
         [  9,  56,  10],
         ...,
         [ 10,  56,  11],
         [ 10,  56,  10],
         [ 10,  55,  10]],

        [[  7,  55,   8],
         [  8,  55,   9],
         [  9,  56,  10],
         ...,
         [ 10,  56,  10],
        

In [6]:
b

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 35, 36,
       37, 38, 39, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 15, 16, 17, 18, 19, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49])

In [8]:
np.sum(c == new_Xtr)

153600

In [14]:
def relabel(new_Xtr, new_Size = 50):
    img_per_cls = new_Size/10
    class_to_swap = 3
    long = int(img_per_cls * class_to_swap)
    temp = copy.deepcopy(new_Xtr[long:int(long +5)][:][:][:])
    cls_to_swp_with = 7
    long_two = int(img_per_cls * cls_to_swp_with)
    new_Xtr[long:int(long +5)][:][:][:] = new_Xtr[long_two : int(long_two+5)][:][:][:]
    new_Xtr[long_two : int(long_two+5)][:][:][:] = temp
    return new_Xtr